<a href="https://colab.research.google.com/github/jaggukaka/TTS/blob/main/experiments/yourtts_telugu_openslr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
!rm -rf /content/TTS
!git clone https://github.com/jaggukaka/TTS

Cloning into 'TTS'...
remote: Enumerating objects: 23501, done.
remote: Counting objects: 100% (4138/4138), done.
remote: Compressing objects: 100% (1329/1329), done.
remote: Total 23501 (delta 2977), reused 3654 (delta 2730), pack-reused 19363
Receiving objects: 100% (23501/23501), 165.99 MiB | 13.83 MiB/s, done.
Resolving deltas: 100% (16972/16972), done.


In [2]:
import torch
torch.cuda.current_device()

0

In [3]:
print (torch.cuda.device_count(), torch.cuda.get_device_name(0), torch.cuda.is_available())
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

1 Tesla P100-PCIE-16GB True
Mon Jan 31 09:23:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    25W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                            

In [4]:
! pwd

/content


In [ ]:
import os
os.chdir('TTS')
!pip install -r requirements.txt
!python setup.py develop


In [49]:
os.chdir('/content/TTS')
! git pull

remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 6 (delta 5), reused 6 (delta 5), pack-reused 0
Unpacking objects: 100% (6/6), done.
From https://github.com/jaggukaka/TTS
   0c6a6784..86df90ee  main       -> origin/main
Updating 0c6a6784..86df90ee
Fast-forward
 TTS/tts/models/vits.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


In [ ]:

os.chdir('/content')
! CUDA_VISIBLE_DEVICES="0" python TTS/TTS/bin/train_tts.py --config_path /content/drive/MyDrive/tts_dubbing_coqui/config_te_new.json --restore_path /content/drive/MyDrive/tts_dubbing_coqui/model_file.pth.tar --coqpit.lr_gen 0.00001

In [ ]:
% load_ext tensorboard


In [ ]:
% tensorboard --logdir /content/drive/MyDrive/tts_dubbing_coqui/checkpoints/VITS-multilingual/tts-telugu/openslr/run_dvectors/vits_tts-telugu-dvectors-January-31-2022_12+26PM-0000000

## The following is the demo for the above trained model. This is for zero-shot VC.

In [54]:
import sys
TTS_PATH = "TTS/"

# add libraries into environment
sys.path.append(TTS_PATH) # set this if TTS is not installed globally

import os
import string
import time
import argparse
import json

import numpy as np
import IPython
from IPython.display import Audio


import torch

from TTS.tts.utils.synthesis import synthesis
from TTS.tts.utils.text.symbols import make_symbols, phonemes, symbols
try:
  from TTS.utils.audio import AudioProcessor
except:
  from TTS.utils.audio import AudioProcessor


from TTS.tts.models import setup_model
from TTS.config import load_config
from TTS.tts.models.vits import *

In [58]:
# model vars 
MODEL_PATH = '/content/drive/MyDrive/tts_dubbing_coqui/checkpoints/VITS-multilingual/tts-telugu/openslr/run_dvectors/vits_tts-telugu-dvectors-January-31-2022_12+26PM-0000000/best_model_442.pth.tar'
CONFIG_PATH = '/content/drive/MyDrive/tts_dubbing_coqui/checkpoints/VITS-multilingual/tts-telugu/openslr/run_dvectors/vits_tts-telugu-dvectors-January-31-2022_12+26PM-0000000/config.json'
TTS_LANGUAGES = "/content/drive/MyDrive/tts_dubbing_coqui/checkpoints/VITS-multilingual/tts-telugu/openslr/run_dvectors/vits_tts-telugu-dvectors-January-31-2022_12+26PM-0000000/language_ids.json"
TTS_SPEAKERS = "/content/drive/MyDrive/tts_dubbing_coqui/speakers_te_pretrained_semodel.json"
USE_CUDA = torch.cuda.is_available()

In [62]:
# load the config
C = load_config(CONFIG_PATH)


# load the audio processor
ap = AudioProcessor(**C.audio)

speaker_embedding = None

C.model_args['d_vector_file'] = TTS_SPEAKERS
C.model_args['use_speaker_encoder_as_loss'] = False

model = setup_model(C)
model.language_manager.set_language_ids_from_file(TTS_LANGUAGES)
# print(model.language_manager.num_languages, model.embedded_language_dim)
# print(model.emb_l)
cp = torch.load(MODEL_PATH, map_location=torch.device('cpu'))
# remove speaker encoder
keys_to_remove = ["speaker_encoder", "audio_transform.kernel"]
model_weights = cp['model'].copy()
for key in list(model_weights.keys()):
  for rkey in keys_to_remove:
    if rkey in key:
      del model_weights[key]


#print ('\n'.join(model_weights.keys()))

model.load_state_dict(model_weights)


model.eval()

if USE_CUDA:
    model = model.cuda()

# synthesize voice
use_griffin_lim = False

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:False
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Using model: vits
Printing the config data  before init_multispeaker VitsConfig(model='vits', run_name='vits_tts-telugu-dvectors', run_description='', epochs=2, batch_size=20, eval_batch_size=30, mixed_precision=False, scheduler_after_epoch=True, run_eval=True, test_delay_epochs=-1,

In [63]:
! pip install -q pydub ffmpeg-normalize

In [64]:
CONFIG_SE_PATH = "/content/drive/MyDrive/tts_dubbing_coqui/config_te_se.json"
CHECKPOINT_SE_PATH = "/content/drive/MyDrive/tts_dubbing_coqui/model_se.pth.tar"

In [65]:
from TTS.tts.utils.speakers import SpeakerManager
from pydub import AudioSegment
from google.colab import files
import librosa

In [66]:
SE_speaker_manager = SpeakerManager(encoder_model_path=CHECKPOINT_SE_PATH, encoder_config_path=CONFIG_SE_PATH, use_cuda=USE_CUDA)

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:512
 | > power:1.5
 | > preemphasis:0.97
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:False
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:False
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:True
 | > db_level:-27.0
 | > stats_path:None
 | > base:10
 | > hop_length:160
 | > win_length:400


In [67]:
def compute_spec(ref_file):
  y, sr = librosa.load(ref_file, sr=ap.sample_rate)
  spec = ap.spectrogram(y)
  spec = torch.FloatTensor(spec).unsqueeze(0)
  return spec

## Below is the Voice conversion code 

In [69]:
print("Select target speaker reference audios files:")
target_files = files.upload()
target_files = list(target_files.keys())
for sample in target_files:
    !ffmpeg-normalize $sample -nt rms -t=-27 -o $sample -ar 16000 -f

Select target speaker reference audios files:


MessageError: ignored